In [67]:
import torch
import torch.nn as nn
from torch.amp import autocast, GradScaler

# 自定义层：基于PyTorch张量操作实现
class CustomLayer(nn.Module):
    def __init__(self, in_features, out_features):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(out_features, in_features))
        self.bias = nn.Parameter(torch.randn(out_features))
    
    def forward(self, x):
        # 自定义操作：矩阵乘法 + 偏置 + 自定义激活
        x = torch.matmul(x, self.weight.t())  # 矩阵乘法
        print(x.dtype)
        x = x + self.bias  # 加偏置
        print(x.dtype)
        x = torch.clamp(x, min=0.01)  # 自定义激活
        print(x.dtype)
        return x

# 测试混合精度是否生效
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device.type)
model = CustomLayer(10, 5).to(device)
input_tensor = torch.randn(32, 10).to(device)

# 混合精度前向传播
with autocast(device.type):
    output = model(input_tensor)
    print("输出张量精度:", output.dtype)  # 输出: torch.float16（默认低精度）

cuda
torch.float16
torch.float32
torch.float32
输出张量精度: torch.float32


In [ ]:
    # 定义模型文件路径
fp32_path = "model_float32.pt"
fp16_path = "model_float16.pt"

# 保存模型为float32精度（默认）
torch.save(model.state_dict(), fp32_path)
print("\n保存float32模型完成")
# 将模型转换为float16并保存
model_float16 = model.half()  # 转换所有参数为float16
torch.save(model_float16.state_dict(), fp16_path)
print("保存float16模型完成")
# 验证保存的模型精度
print("\n验证保存的模型精度:")
state_fp32 = torch.load(fp32_path)
print(f"float32模型权重精度: {state_fp32['weight'].dtype}")
state_fp16 = torch.load(fp16_path)
print(f"float16模型权重精度: {state_fp16['weight'].dtype}")

# 测试加载float16模型进行推理
model_loaded = CustomLayer(10, 5).to(device)
model_loaded.load_state_dict(torch.load(fp16_path))
model_loaded.half()  # 确保模型处于float16模式 
#无论load32还是16的模型，这儿转成了half，后面就要加autocast，否则维度不匹配



保存float32模型完成
保存float16模型完成

验证保存的模型精度:
float32模型权重精度: torch.float32
float16模型权重精度: torch.float16


CustomLayer()

In [69]:
with autocast(device.type):
    output_loaded = model_loaded(input_tensor)
    print(f"\n加载的float16模型输出精度: {output_loaded.dtype}")

torch.float16
torch.float16
torch.float16

加载的float16模型输出精度: torch.float16


In [70]:
import os

# 无论程序是否正常执行，都删除保存的模型文件
if os.path.exists(fp32_path):
    os.remove(fp32_path)
    print(f"\n已删除 {fp32_path}")
if os.path.exists(fp16_path):
    os.remove(fp16_path)
    print(f"已删除 {fp16_path}")
    


已删除 model_float32.pt
已删除 model_float16.pt
